In [5]:
import numpy as np
import pandas as pd
import math as math
import cvxpy as cp
from sklearn.metrics import accuracy_score as acc_s

print("start")

start


In [6]:
#read the training data
rd = pd.read_csv('./train.csv',header = None)
#print(rd)
def initial ():
    #first 4000 as training data, rest as validation
    #first column for y, rest for x
    train_x = rd.iloc[:4000,1:].to_numpy()
    train_y = rd.iloc[:4000,0].to_numpy()

    vali_x = rd.iloc[4000:,1:].to_numpy()
    vali_y = rd.iloc[4000:,0].to_numpy()

    # change the first row 0 to -1
    train_y = np.where(train_y == 0, -1 , train_y)
    vali_y = np.where(vali_y == 0, -1, vali_y)
    return train_x, train_y, vali_x, vali_y

data_set = initial()
train_x = data_set[0]
train_y = data_set[1]
vali_x = data_set[2]
vali_y = data_set[3]

print(train_x)
print(train_x.shape)
print("-----------")
print(train_y)
print(train_y.shape)
print("-----------")
print(vali_x)
print(vali_x.shape)
print("-----------")
print(vali_y)
print(vali_y.shape)

[[-0.36 -0.91 -0.99 ...  0.3   2.44 -1.26]
 [-1.4  -1.9   0.09 ... -0.2  -0.92 -0.46]
 [-0.43  1.45 -0.68 ...  0.12  0.01 -0.56]
 ...
 [-0.57  0.14 -0.62 ...  0.    0.38 -0.82]
 [ 0.4   0.16 -0.49 ...  0.89  0.21  1.09]
 [ 0.86 -0.23 -2.01 ...  0.21  0.68  2.49]]
(4000, 200)
-----------
[-1.  1.  1. ...  1.  1.  1.]
(4000,)
-----------
[[-0.26  0.66  0.31 ... -1.79  1.48  0.01]
 [ 0.47 -0.01 -0.5  ... -0.19  1.84 -0.54]
 [ 0.39  1.57 -1.69 ... -0.11  2.06 -0.55]
 ...
 [ 1.18  0.78 -0.02 ... -0.63  0.29 -0.96]
 [-0.55  1.16 -0.56 ...  2.97 -2.66 -1.9 ]
 [ 0.8  -0.97 -0.15 ... -1.58  0.35  1.06]]
(4500, 200)
-----------
[-1.  1. -1. ...  1.  1. -1.]
(4500,)


In [12]:
# Q2

def svm_train_primal ( data_train , label_train , regularisation_para_C ):
    m = data_train.shape[0]
    n = data_train.shape[1]

    #construct variable
    w = cp.Variable(n)
    b = cp.Variable()
    epsilon = cp.Variable(m)
    
    #construct problem
    ob = cp.Minimize((1/2) * cp.sum_squares(w) + (regularisation_para_C/m) * cp.sum(epsilon))
    # "*" for scalar, "@" for vector , "multiply(a,b)" for matrix
    constraints = [cp.multiply(label_train,(data_train @ w + b)) + epsilon >= 1, epsilon >= 0]
    problem = cp.Problem(ob,constraints)
    problem.solve()

    return w.value, b.value, epsilon.value

def svm_predict_primal ( data_test , label_test , svm_model ):
    #np.sign give the classification -1 or 1
    return acc_s(label_test, np.sign(np.dot(data_test, svm_model[0]) + svm_model[1]))

svm_model = svm_train_primal(train_x,train_y,100)
W = svm_model[0]
b = svm_model[1]

print("sum of all dimensions of W is " + str(np.sum(W)))
print("b is "+ str(b))

test_accuracy = svm_predict_primal (vali_x, vali_y, svm_model)
print("The accuracy verified by the validation data: " + str(test_accuracy))

sum of all dimensions of W is -0.14616662483607457
b is 1.7797371047962776
The accuracy verified by the validation data: 0.9695555555555555


In [17]:
#Q3
def svm_train_dual (data_train, label_train, regularisation_para_C):
    m = data_train.shape[0]
    n = data_train.shape[1]
    
    #construct variable
    alpha = cp.Variable(m)

    #define dual problem
    # "*" for scalar, "@" for vector , "multiply(a,b)" for matrix
    ob = cp.Maximize(cp.sum(alpha) - (1 / 2) * cp.sum_squares(cp.multiply(alpha, label_train) @ data_train))
    constraint = [0 <= alpha, alpha <= regularisation_para_C/m, cp.sum(cp.multiply(alpha,label_train)) == 0]
    
    problem = cp.Problem(ob, constraint)
    problem.solve()

    return alpha.value

alpha = svm_train_dual(train_x, train_y, 100)

print("sum of all dimensions of optimal alpha: " + str(np.sum(alpha)))

sum of all dimensions of optimal alpha: 7.2912583245569005


In [21]:
#Q4
w_star = (cp.multiply(alpha, train_y) @ train_x).value
w_star_sum = np.sum(w_star)
b_star = train_y - (train_x @ w_star)

print(w_star)
print(w_star_sum)
print(np.mean(b_star))

[-2.55939591e-02 -1.14007921e-01  4.32855212e-02 -5.18819300e-02
  7.53472963e-02 -7.84397544e-03  2.65824264e-02 -9.45503426e-03
  4.40336593e-03  1.58195680e-03  9.38970723e-02  6.42173161e-02
  6.94519223e-02  6.27878091e-02  1.89513082e-02 -2.33295766e-02
 -5.83849380e-02 -4.54946031e-02 -5.19864228e-02 -2.69134823e-02
 -1.74617965e-03  4.38137199e-02 -8.54178243e-03 -4.56328690e-02
 -1.88722860e-02 -1.86577877e-02 -4.35992174e-03  1.05853557e-02
 -7.25165256e-02  1.13775842e-01 -2.23632064e-02 -1.37427911e-02
  5.99524516e-02 -2.08665832e-02  8.54609108e-02 -3.98405560e-02
  2.32234759e-02 -5.28302006e-02  5.45850507e-02 -3.69796115e-02
  2.74815343e-02  2.24402462e-02 -2.34412061e-02 -5.06717780e-03
  2.71047146e-02 -2.43686942e-02  1.45912621e-01  7.18292455e-02
  1.47053334e-02  9.80263003e-02 -3.10295884e-02  3.86233566e-02
  8.62364617e-02 -4.83414914e-02  5.33736064e-02 -4.19834603e-02
 -5.54077372e-02  9.72250016e-02 -3.71558817e-02  7.25102735e-01
  2.34932984e-02 -4.94497